In [54]:
import numpy as np
import scipy as sp
from scipy.stats import gamma
import pandas as pd
import matplotlib.pyplot as plt
import math

Question 1: Nous cherchons une équation pour la loi à posteriori $\pi(\lambda|X=x)$ ainsi que f(x). L'énnoncé du travail dicte que la plausabilité de cette loi peut se trouver de cette façon: 
$$\pi(\lambda|X=x)=\frac{1}{f(x)}\frac{\lambda^{\alpha-1}e^{-\beta\lambda}B^{\alpha}}{\Gamma(\alpha)}\Pi_{x_i=1}^{T} \lambda e^{-\lambda x_i}$$
F(x) correspond à la constante de normalisation nécéssaire. Autrement dit, la probabilité de trouver une valeur à cette équation sur le domaine des $\lambda$ doit être de 1, il est donc possible de réécrire: 
$$1=\int_0^\infty\bigg[\frac{1}{f(x)}\frac{\lambda^{\alpha-1}e^{-\beta\lambda}B^{\alpha}}{\Gamma(\alpha)}\Pi_{x_i=1}^{T} \lambda e^{-\lambda x_i}\bigg]d\lambda$$
$$f(x)=\frac{B^{\alpha}}{\Gamma(\alpha)}\Pi_{x_i=1}^{T}e^{x_i}\int_0^\infty\bigg[\lambda^{\alpha-1}e^{-\beta\lambda} \lambda^T e^{-T\lambda}\bigg]d\lambda$$

À SIMPLIFIER PLUS TARD, IL Y A DES ERREURS DANS LE DÉV. MAIS PAS ENCORE EU LE TEMPS DE CORRIGER. Je crois toutefois que la démarche ok. 

Question 2:
$$\lambda_0=\int_0^\infty \pi(\lambda)d\lambda=\int_0^\infty \lambda^{\alpha-1}e^{-\beta \lambda}\beta^\lambda d\lambda$$

À CONTINUER




Question 3: Pour lire les trois fichiers, il est possible d'utiliser le module pandas. 

In [87]:

data0=pd.read_csv(f"Tp1/data/activite_temps_0.txt", sep=" ", header=None)
data1=pd.read_csv(f"Tp1/data/activite_temps_1.txt", sep=" ", header=None)
data2=pd.read_csv(f"Tp1/data/activite_temps_2.txt", sep=" ", header=None)

Question 4: En reprenant l'équation pour f(x) écrite plus haut, avec $\alpha=2$, $\beta=0.5$, $0\leq \lambda \leq 200$ et $T=97$: 

$$f(x)=\int_0^{200}\bigg[\frac{\lambda^1e^{-\frac{\lambda}{4}}B^{2}}{\Gamma(2)}\Pi_{x_i=1}^{97} \lambda e^{-\lambda x_i}\bigg]d\lambda$$
Pour se simplifier la tâche lors de la construction de cette équation dans python, nous pouvons y aller en plusieurs morceaux. Premièrement, le terme $\frac{\lambda^1e^{-\frac{\lambda}{4}}B^{2}}{\Gamma(2)}$ peut être représenté comme un fonction gamma avec les paramètres $\alpha=2$,$\beta=\frac{1}{4}$ grâce au module scipy. Le produit $\Pi_0^{97}e^{-\lambda x_i}= e^{-\lambda \cdot \sum_0^{97} -x_i}$. Il ne reste qu'à itérer sur les valeurs de X pour calculer le paramètre. 

In [113]:


def simpson(f, a, b, N):
    h = (b - a) / N
    x = np.linspace(a, b, N+1)
    y = f(x)
    return h/3 * (y[0] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-1:2]) + y[-1])

def f(x):
    alpha=2
    beta=1/4
    dataset=data0
    return(math.e**(x*pd.DataFrame.sum(dataset))*gamma.pdf(x, alpha, loc=0, scale=1/beta))


a, b, n = 0, 1, 4
result = simpson(f, a, b, n)
print(result)


ValueError: Length of values (5) does not match length of index (1)